# Features Extraction with open LLMs

The present notebook uses Hugging Face libraries and models (specificly Llama 3.2 Instruct 1B) running in T4 Google Colab GPU to extract information from text.

The data consists of 21k job description (job ads from HrFlow.ai) with an average of 1k tokens per job description.

The goal is to extract the location, experience level, company name and the industry of the company for each job ad.

The task was completed in 3 hours with around 80M tokens of input with near maximum GPU memory utilization.

In [ ]:
import os
import ast
import re
import json
from IPython.display import display, HTML
import numpy as np
import pandas as pd
import time
import gc
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



# HF_TOKEN = ""
# os.environ["HUGGING_FACE_HUB_TOKEN"] = HF_TOKEN

In [ ]:
# X_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Challenge/data/x_train_Meacfjr.csv", delimiter=",", quotechar='"')
# y_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Challenge/data/y_train_SwJNMSu.csv", delimiter=",", quotechar='"')

# # Convert 'job_ids' and 'actions' from strings to actual lists
# X_train["job_ids"] = X_train["job_ids"].apply(ast.literal_eval)
# X_train["actions"] = X_train["actions"].apply(ast.literal_eval)

with open("/content/drive/MyDrive/Colab Notebooks/Challenge/data/job_listings.json", "r") as file:
    jobs = json.load(file)

# 1. Data Preprocessing

## 1.1 Split Job Description

In [ ]:
def extract_info(job_id, text):

    job_description = text
    main = text.split("SECTION")[0]

    # Extract title
    title_match = re.search(r"TITLE\n(.+)", text)
    title = title_match.group(1).strip() if title_match else None

    # Extract summary
    summary_match = re.search(r"SUMMARY\n([\s\S]+?)(?:\nSECTION:|$)", text)
    summary = summary_match.group(1).strip() if summary_match else None

    # Extract descriptions
    descriptions = re.findall(r"SECTION:\nDESCRIPTION: ([\s\S]+?)(?:\nSECTION:|$)", text)

    # Extract skills only from the SKILLS section
    skills_match = re.findall(r"SKILLS\n(?:- (\{.*?\})\n?)+", text)
    skills = [
        ast.literal_eval(skill)["name"]
        for match in skills_match for skill in re.findall(r"- (\{.*?\})", match)
        if "'name':" in skill
    ]

    # Extract tasks only from the TASKS section
    tasks_match = re.findall(r"TASKS\n(?:- (\{.*?\})\n?)+", text)
    tasks = [
        ast.literal_eval(task)["name"]
        for match in tasks_match for task in re.findall(r"- (\{.*?\})", match)
        if "'name':" in task
    ]

    # Extract languages, certifications (currently empty sections)
    languages = []
    certifications = []

    # Extracted data
    row = {
        "job_id": int(job_id),
        "title_section": title,
        # "summary": summary,
        # "descriptions": "\n\n".join(descriptions),
        # "languages": "\n".join(languages),
        # "certifications": "\n".join(certifications),
        # "skills": "\n".join(skills),
        # "tasks": "\n".join(tasks),
        "job_description": job_description,
        # "main": main,
        # "embedding": []
    }
    return row

In [ ]:
rows = []
for job_id in jobs.keys():
    row = extract_info(job_id, jobs[job_id])
    rows.append(row)
df_jobs = pd.DataFrame(rows)

print(df_jobs.shape)
print(df_jobs.info())

(21917, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21917 entries, 0 to 21916
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_id           21917 non-null  int64 
 1   title_section    21913 non-null  object
 2   job_description  21917 non-null  object
dtypes: int64(1), object(2)
memory usage: 513.8+ KB
None


# 1.2 Create Features - title, location, seniority, summary, company, industry, contract type, linguas exigidas

## GPU Requirements

In [ ]:
!nvcc --version # find the CUDA driver build above

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Mar  9 16:50:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
!pip install vllm accelerate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.9/396.9 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

##Test llama 3.2 3B

In [ ]:
from vllm import SamplingParams, LLM

In [ ]:
# Load model into GPU memory
llm = LLM(
    model="unsloth/Llama-3.2-1B-Instruct",
    dtype="float16",      # Use bfloat16 if you are using better GPU than T4
    max_model_len=4096,
    task="auto")

INFO 03-09 16:52:39 __init__.py:207] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

WARNING 03-09 16:52:40 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 03-09 16:53:07 config.py:549] This model supports multiple tasks: {'reward', 'embed', 'classify', 'generate', 'score'}. Defaulting to 'generate'.
INFO 03-09 16:53:07 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='unsloth/Llama-3.2-1B-Instruct', speculative_config=None, tokenizer='unsloth/Llama-3.2-1B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

...

INFO 03-09 16:53:29 weight_utils.py:270] Time spent downloading weights for unsloth/Llama-3.2-1B-Instruct: 16.866020 seconds
INFO 03-09 16:53:29 weight_utils.py:304] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 03-09 16:53:33 model_runner.py:1115] Loading model weights took 2.3185 GB
INFO 03-09 16:53:35 worker.py:267] Memory profiling takes 1.39 seconds
INFO 03-09 16:53:35 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 03-09 16:53:35 worker.py:267] model weights take 2.32GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.19GiB; the rest of the memory reserved for KV Cache is 9.72GiB.
INFO 03-09 16:53:35 executor_base.py:111] # cuda blocks: 19896, # CPU blocks: 8192
INFO 03-09 16:53:35 executor_base.py:116] Maximum concurrency for 4096 tokens per request: 77.72x
INFO 03-09 16:53:38 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_uti

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:35<00:00,  1.01s/it]

INFO 03-09 16:54:14 model_runner.py:1562] Graph capturing finished in 35 secs, took 0.12 GiB
INFO 03-09 16:54:14 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 40.58 seconds


In [ ]:
df_test = df_jobs.sample(100)

In [ ]:
sampling_params = SamplingParams(
    max_tokens=10,  # Limit output length
    temperature=0.0  # Make results deterministic
)

# 🌟 Example batch of prompts
prompts = [f'Here is the job description: \n\n \"\"\"{j}\"\"\" \n\n Return ONLY the location in the format "city, country" or "No location specifiec" otherwise.\n\n Response: ' for j in df_test["job_description"]]

# 🚀 Run batch inference (processed in parallel)
start_time = time.time()
outputs = llm.generate(prompts, sampling_params)  # increase tensor_parallel_size=2 if using better GPU


print("\nProcess time of 100 job descriptions:", time.time() - start_time)

Around 6 job descriptions or 6k tokens per second. Resulting hopefully in 1h10min per feature extraction.

In [ ]:
# Manual truncation after generation using stop sequences
def clean_response(response):
    stop_chars = ['\n', '.', 'The']  # Stop at newline, period, or "The"
    for char in stop_chars:
        if char in response:
            return response.split(char)[0].strip()  # Keep the first part
    return response.strip()  # Trim spaces and return

processed_responses = [clean_response(output.outputs[0].text) for output in outputs]

# Print results
for i, res in enumerate(processed_responses):
    print(f"Response {i+1}: {res}")

## Create Features

In [ ]:
# Define Features
features = ['location', 'seniority', 'company', 'industry']
for feature in features:
    df_jobs[feature] =  pd.NA

In [ ]:
# Define feature-specific prompt creation and response cleaning functions
def create_location_prompt(job_desc):
    return f'Here is the job description: \n\n \"\"\"{job_desc}\"\"\" \n\n Return ONLY the location in the format "city, country" or "No location specifiec" otherwise.\n\n Response: '

def create_seniority_prompt(job_desc):
    return f'Here is the job description: \n\n \"\"\"{job_desc}\"\"\" \n\n \
    The senirioty level of the job is either Intern, Junior, Mid, Senior or Direction/Executive.\n\n \
    Return ONLY the seniority level of this job among the options. Return "Not Specified" if it was not specified in the job ad.\n\n Response :'

def create_company_prompt(job_desc):
    return f'Here is the job description: \n\n \"\"\"{job_desc}\"\"\" \n\n \
    Return ONLY the name of the company. \n\n Response: '

def create_industry_prompt(job_desc):
    return f'Here is the job description: \n\n \"\"\"{job_desc}\"\"\" \n\n \
    Return ONLY the industry sector of the company. \n\n Response: '

def clean_response(response):
    stop_chars = ['\n', '.', 'The']
    for char in stop_chars:
        if char in response:
            return response.split(char)[0].strip()
    return response.strip()

def clean_location_response(response):
    return clean_response(response)

def clean_seniority_response(response):
    return clean_response(response)

def clean_company_response(response):
    return clean_response(response)

def clean_industry_response(response):
    return clean_response(response)

# Map feature to its prompt and clean function
feature_prompts = {
    'location': create_location_prompt,
    'seniority': create_seniority_prompt,
    'company': create_company_prompt,
    'industry': create_industry_prompt
}

feature_cleaners = {
    'location': clean_location_response,
    'seniority': clean_seniority_response,
    'company': clean_company_response,
    'industry': clean_industry_response
}

In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer for Llama 3.2 Instruct 1B
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

def truncate_prompt(prompt, max_tokens=4000):
    """
    Truncates the input prompt to ensure it does not exceed the max token limit.
    Uses Hugging Face tokenizer to tokenize the prompt and truncate it accordingly.
    """
    # Tokenize the prompt and ensure it doesn't exceed the max token length
    encoded = tokenizer(prompt, truncation=True, max_length=max_tokens)
    return tokenizer.decode(encoded['input_ids'])

In [ ]:
output_file = "/content/drive/MyDrive/Colab Notebooks/Challenge/data/job_features.csv"

def create_features(df, batch_size, save_every=10):
    for feature in features:

        # Split the DataFrame into smaller chunks for batch processing
        chunks = np.array_split(df['job_description'], len(df) // batch_size + 1)

        # Process each chunk (batch)
        for i, chunk in enumerate(chunks):
            # Create batch prompts for the current chunk
            batch_prompts = [truncate_prompt(feature_prompts[feature](j)) for j in chunk]

            print(f"Processing batch {i + 1}/{len(chunks)} with batch size: {len(batch_prompts)}")

            # Run batch inference on GPU
            start_time = time.time()
            outputs = llm.generate(batch_prompts, sampling_params)  # Run LLM generation
            end_time = time.time()

            # Clean the responses and store them in the feature column
            processed_responses = [feature_cleaners[feature](output.outputs[0].text) for output in outputs]

            # Store the results back into the dataframe (use an offset to update the corresponding rows)
            df.loc[chunk.index, feature] = processed_responses

            print(f"Feature '{feature}' processed in {i + 1} batch(s) in {end_time - start_time:.2f} seconds")

            # Save periodically after every `save_every` batches
            if (i + 1) % save_every == 0:
                print(f"Saving...")
                df.to_csv(output_file, index=False)  # Save to CSV (or use df.to_pickle() for faster saving)

            # Clear GPU cache and collect garbage after each batch to optimize memory
            torch.cuda.empty_cache()  # Clear PyTorch cache
            gc.collect()  # Run garbage collection

        print(f"Completed feature {feature}.\nSaving...\n\n")
        df.to_csv(output_file, index=False)

    return df

In [ ]:
df_jobs = create_features(df_jobs, batch_size=128)

Processing batch 1/172 with batch size: 128


Processed prompts: 100%|██████████| 128/128 [00:25<00:00,  4.94it/s, est. speed input: 7484.63 toks/s, output: 21.59 toks/s]


Feature 'seniority' processed in 1 batch(s) in 26.36 seconds
Processing batch 2/172 with batch size: 128


Processed prompts: 100%|██████████| 128/128 [00:21<00:00,  5.88it/s, est. speed input: 8154.03 toks/s, output: 18.89 toks/s]


Feature 'seniority' processed in 2 batch(s) in 22.29 seconds
Processing batch 3/172 with batch size: 128


...

Feature 'industry' processed in 170 batch(s) in 13.30 seconds
Saving...
Processing batch 171/172 with batch size: 127


Processed prompts: 100%|██████████| 127/127 [00:11<00:00, 11.22it/s, est. speed input: 8183.34 toks/s, output: 69.07 toks/s]


Feature 'industry' processed in 171 batch(s) in 11.59 seconds
Processing batch 172/172 with batch size: 127


Processed prompts: 100%|██████████| 127/127 [00:09<00:00, 13.58it/s, est. speed input: 7952.78 toks/s, output: 82.67 toks/s]


Feature 'industry' processed in 172 batch(s) in 9.57 seconds
Completed feature industry.
Saving...


